In [ ]:
# Kutuphaneler ve cikti sayisi ayarlama kismi
import pandas as pd
import numpy as np
import random

# -----------------------  O  ---------------------------- #

# Cikti dosya almak icin
from google.colab import files

# -----------------------  O  ---------------------------- #

ciktiSayi = 10000

In [ ]:
# Kullanilacak degiskenler ve kategor'k karsiliklari
mekanlar = ['Konut', 'IsYeri', 'AcikAlan', 'KamuBinasi', 'Ibadethane', 'TopluTasima']
vakitler = ['Gunduz', 'Gece']
kilitDurumlari = ['KilitliDegil', 'KilitKirilarak', 'AnahtarUydurarak', 'BilisimYoluyla']
yontemler = ['BedenGucu', 'OzelBeceri', 'TeknolojiKullanimi']
tamamlanmaDurumlari = ['Tamamlandi', 'Tesebbus']
maddiDurumlar = ['Dusuk', 'Orta', 'Yuksek']
adliSiciller = ['Sabikasiz', 'Tekerrur']
hakimTakdirleri = ['IyiHal', 'KotuHal']

akilSaglikDurumlari = ['Yerinde', 'Zayiflik', 'Hastalik']

sanikTutumlari = ['Pisman', 'Inkar', 'Sessiz', 'Celiski', 'Agresif']
avukatStratejileri = ['DelilYetersizligi', 'VasifDegisikligi', 'IndirimOdakli', 'AkliDenge', 'SahsiCezasizlik']


In [ ]:
def olusturSucOzellikleri():
    # Sucun ozelliklerinin belirlendigi kisim
    row = {}
    # Degisken kismindan mekan ve zamani rastgeleden cekiyoruz
    row['sucMekani'] = random.choice(mekanlar)
    row['sucVakti'] = random.choice(vakitler)

    if row['sucMekani'] == 'AcikAlan':
        row['kilitDurumu'] = 'KilitliDegil'
    else:
        row['kilitDurumu'] = random.choice(kilitDurumlari)

    # -----------------------  O  ---------------------------- #

    # Malın degeri (agirli sekilde ayarladim cunku gunluk hayata uymasi gerekli)
    sans = random.random()
    if sans < 0.20: row['malinDegeriTL'] = random.randint(50, 1000)
    elif sans < 0.70: row['malinDegeriTL'] = random.randint(1001, 30000)
    elif sans < 0.90: row['malinDegeriTL'] = random.randint(30001, 100000)
    else: row['malinDegeriTL'] = random.randint(100001, 500000)

    # Malin degerini kategoriye indiriyoruz
    if row['malinDegeriTL'] < 2000: row['degerKategorisi'] = 'Hafif'
    elif row['malinDegeriTL'] < 50000: row['degerKategorisi'] = 'Normal'
    else: row['degerKategorisi'] = 'Yuksek'

    # -----------------------  O  ---------------------------- #

    # Bu kismi da rastgeleden cekiyoruz
    row['sucYontemi'] = random.choice(yontemler)
    row['sucTamamlanma'] = random.choice(tamamlanmaDurumlari)

    return row

In [ ]:
def olusturFailProfili():
    # Sahsin karakterinin olusturuldugu kisim

    row = {}
    row['failYasi'] = random.randint(10, 70)
    row['akilSagligi'] = np.random.choice(akilSaglikDurumlari, p=[0.94, 0.05, 0.01])

    # -----------------------  O  ---------------------------- #

    # Sahsin 12-15 yas arasinda ise akli dengesi var yok kontrol etmemeiz lazim
    if 12 <= row['failYasi'] < 15:
        row['algilamaYetenegi'] = np.random.choice(['Var', 'Yok'], p=[0.8, 0.2])
    else:
        row['algilamaYetenegi'] = 'Uygulanmaz'

    # -----------------------  O  ---------------------------- #

    # Kamu gorevini kotuye kullanma TCK'da cok agur bir suc oldugu icin gorevli mi ve kotuye mi kullandi kontrol kismi
    if row['failYasi'] >= 18:
        row['kamuGorevlisiMi'] = np.random.choice(['Evet', 'Hayir'], p=[0.08, 0.92])
    else:
        row['kamuGorevlisiMi'] = 'Hayir'

    # -----------------------  O  ---------------------------- #
    if row['kamuGorevlisiMi'] == 'Evet':
        row['goreviKotuyeKullanma'] = 'Evet' if random.random() < 0.3 else 'Hayir'
    else:
        row['goreviKotuyeKullanma'] = 'Hayir'

    # -----------------------  O  ---------------------------- #

    row['maddiDurum'] = random.choice(maddiDurumlar)
    row['adliSicil'] = random.choice(adliSiciller)
    row['orgutBaglantisi'] = np.random.choice(['Evet', 'Hayir'], p=[0.05, 0.95])

    return row

In [ ]:
# Cezai ehliyet hesaplama kismi

def belirleCezaiEhliyet(row):

    yas = row['failYasi']
    akil = row['akilSagligi']
    algilama = row['algilamaYetenegi']

    # -----------------------  O  ---------------------------- #

    if yas < 12:
        return 'Yok'
    elif akil == 'Hastalik':
        return 'Yok'
    elif 12 <= yas < 15 and algilama == 'Yok':
        return 'Yok'

    # -----------------------  O  ---------------------------- #

    elif 12 <= yas < 15 and algilama == 'Var':
        return 'Indirimli'
    elif 15 <= yas < 18:
        return 'Indirimli'
    elif akil == 'Zayiflik':
        return 'Indirimli'
    else:
        return 'Tam'

In [ ]:
# Ceza hesaplama kismi

def hesaplaCezaVeInfaz(row):

    # Cezai ehliyet yoksa direkt 0 vermeliyiz
    if row['cezaiEhliyetDurumu'] == 'Yok':
        row['hapisCezasiAy'] = 0
        row['paraCezasiTL'] = 0
        row['cezaDurumu'] = 'GuvenlikTedbiri' # Çocuklara veya akıl hastalarına özgü güvenlik tedbiri
        row['infazRejimi'] = 'Yok'
        row['infazSuresiAy'] = 0
        row['yatarSuresiAy'] = 0
        return row

    # -----------------------  O  ---------------------------- #

    baseCeza = 12 # Baz ceza 12 aydan basliyor

    # Cezayi arttiran unsurlar
    if row['sucVakti'] == 'Gece': baseCeza *= 1.5
    if row['kilitDurumu'] != 'KilitliDegil': baseCeza *= 1.3
    if row['kamuGorevlisiMi'] == 'Evet' and row['goreviKotuyeKullanma'] == 'Evet': baseCeza *= 1.5
    if row['orgutBaglantisi'] == 'Evet': baseCeza *= 2.0
    if row['adliSicil'] == 'Tekerrur': baseCeza += 6

    # Cezayi azaltan unsurlar
    if row['sucTamamlanma'] == 'Tesebbus': baseCeza *= 0.5
    if row['degerKategorisi'] == 'Hafif': baseCeza *= 0.5
    if row['zararGiderildiMi'] == 'Evet': baseCeza *= 0.4

    # TCK31 ve TCK32 (yas ve cezai ehliyet)
    yas = row['failYasi']
    if row['cezaiEhliyetDurumu'] == 'Indirimli':
        if 12 <= yas < 15:
            baseCeza *= 0.5 # TCK 31/2: Yarı oranında indirim
        elif 15 <= yas < 18:
            baseCeza *= 0.67 # TCK 31/3: 1/3 oranında indirim (kalan 2/3)
        elif row['akilSagligi'] == 'Zayiflik':
            baseCeza *= 0.75 # TCK 32/2: Takdiren 1/4 indirim

    # TCK62 Iyi hal
    if row['hakimTakdiri'] == 'IyiHal': baseCeza -= (baseCeza / 6)
    if row['sanikTutumu'] == 'Inkar' and row['hakimTakdiri'] == 'KotuHal': baseCeza *= 1.1

    finalHapisAy = int(round(baseCeza))

    # -----------------------  O  ---------------------------- #

    # Infaz ve yatar
    row['cezaDurumu'] = 'Hapis'
    row['hapisCezasiAy'] = finalHapisAy
    row['paraCezasiTL'] = 0
    row['infazRejimi'] = 'Yok'
    row['infazSuresiAy'] = 0
    row['yatarSuresiAy'] = 0

    if finalHapisAy == 0:
        row['cezaDurumu'] = 'Beraat'
    elif finalHapisAy < 24 and row['adliSicil'] == 'Sabikasiz':
        row['cezaDurumu'] = 'HAGB'
    elif finalHapisAy <= 12 and row['maddiDurum'] != 'Dusuk':
        row['cezaDurumu'] = 'ParaCezasi'
        gunluk = 50 if row['maddiDurum'] == 'Orta' else 100
        row['paraCezasiTL'] = finalHapisAy * 30 * gunluk
        row['hapisCezasiAy'] = 0
    else:
        row['cezaDurumu'] = 'Hapis'
        if row['adliSicil'] == 'Tekerrur':
            infaz = finalHapisAy * (2/3)
            row['infazRejimi'] = 'Mukerrir'
        else:
            infaz = finalHapisAy / 2
            row['infazRejimi'] = 'Standart'

        row['infazSuresiAy'] = int(infaz)
        yatar = infaz - 12
        if yatar < 0: yatar = 0
        row['yatarSuresiAy'] = int(yatar)

    return row

In [ ]:
def anaVeriUretici(n):
    dataset = []

    for i in range(n):
        sucRow = olusturSucOzellikleri()
        failRow = olusturFailProfili()

        # Sözlük birleştirme
        row = {**sucRow, **failRow}

# -----------------------  O  ---------------------------- #

        # Cezai Ehliyeti, diğer hesaplamalardan önce belirle
        row['cezaiEhliyetDurumu'] = belirleCezaiEhliyet(row)

        row['zararGiderildiMi'] = random.choice(['Evet', 'Hayir'])
        row['hakimTakdiri'] = random.choice(hakimTakdirleri)

        # Sanık tutumu uyumlu olmasi ve IRL'e uygun olmasi icin oransal olarak
        if row['zararGiderildiMi'] == 'Evet':
            row['sanikTutumu'] = np.random.choice(['Pisman', 'Sessiz'], p=[0.9, 0.1])
        elif row['hakimTakdiri'] == 'IyiHal':
            row['sanikTutumu'] = 'Pisman'
        else:
            row['sanikTutumu'] = np.random.choice(['Inkar', 'Celiski', 'Agresif'], p=[0.6, 0.2, 0.2])

        # Avukat stratejisi
        if row['cezaiEhliyetDurumu'] == 'Yok' or row['akilSagligi'] != 'Yerinde':
            row['avukatStratejisi'] = 'AkliDenge' # Avukat direkt ehliyetsizlikten yürür
        elif row['failYasi'] < 18:
            row['avukatStratejisi'] = 'IndirimOdakli' # Yaş indirimine oynar
        elif row['kilitDurumu'] == 'KilitliDegil' and row['degerKategorisi'] == 'Hafif':
            row['avukatStratejisi'] = 'VasifDegisikligi'
        elif row['adliSicil'] == 'Sabikasiz':
            row['avukatStratejisi'] = 'IndirimOdakli'
        else:
            row['avukatStratejisi'] = 'DelilYetersizligi'

# -----------------------  O  ---------------------------- #

        # Cezayı Hesapla
        row = hesaplaCezaVeInfaz(row)
        dataset.append(row)

    return pd.DataFrame(dataset)

In [ ]:
dfFinal = anaVeriUretici(ciktiSayi)
dfFinal.to_csv('hirsizlikVeriSetiFinalTRAIN.csv', index=False)
files.download('hirsizlikVeriSetiFinalTRAIN.csv')
print(dfFinal.head(10))

# -----------------------  O  ---------------------------- #

# ^^_ Sunguru 02:50 10.12.2025

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

     sucMekani sucVakti       kilitDurumu  malinDegeriTL degerKategorisi  \
0        Konut   Gunduz    KilitKirilarak          69565          Yuksek   
1       IsYeri   Gunduz    KilitKirilarak          19509          Normal   
2       IsYeri   Gunduz  AnahtarUydurarak          22198          Normal   
3     AcikAlan   Gunduz      KilitliDegil          92760          Yuksek   
4     AcikAlan   Gunduz      KilitliDegil          52190          Yuksek   
5        Konut     Gece  AnahtarUydurarak          20941          Normal   
6  TopluTasima     Gece    BilisimYoluyla            318           Hafif   
7   Ibadethane   Gunduz    BilisimYoluyla          37268          Normal   
8     AcikAlan   Gunduz      KilitliDegil            710           Hafif   
9   KamuBinasi   Gunduz  AnahtarUydurarak          13692          Normal   

           sucYontemi sucTamamlanma  failYasi akilSagligi algilamaYetenegi  \
0          OzelBeceri    Tamamlandi        65     Yerinde       Uygulanmaz   
1  Tekn